In [1]:
import pandas as pd
import pandas_profiling
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, HTML
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns
import calendar
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_columns', None)  
from wordcloud import WordCloud, STOPWORDS
import folium
from folium import plugins

# 1. Merge datasets

In [2]:
data_location = '/Users/anamaria/Downloads/NUSE 934 611(M) 2017-2018.dsv'
data2018=pd.read_csv(data_location,delimiter="|")

/Users/anamaria/Desktop/dev/security_project/security_venv/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning:

Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.



In [3]:
data_location = '/Users/anamaria/Downloads/NUSE 934-611-611M ENERO2019.csv'
data2019=pd.read_csv(data_location,delimiter=";")

In [111]:
frames = [data2018, data2019]
data = pd.concat(frames)
merged_nuse = data.loc[data['TIPO_DETALLE'] == '934 - RIÑA']

In [112]:
merged_nuse

,STR_NUMERO_INTERNO,FECHA,HORA,ANIO,MES,COD_LOCALIDAD,LOCALIDAD,COD_UPZ,UPZ,COD_SEC_CATAST,SEC_CATASTRAL,COD_BARRIO,BARRIO,TIPO_UNICO,TIPO_DETALLE,LATITUD,LONGITUD,STR_DIRECCION_INCIDENTE,ESTADO_INCIDENTE,PERIODO_TS,LOG_TEXT
0,NU10_127625172,02-MAR-17,3,2017,3,15,ANTONIO NARIÑO,UPZ38,RESTREPO,002107,LA FRAGUA,3301,LA_FRAGUA,934,934 - RIÑA,4.592684,-74.106371,KR 28 15A SUR 25,NaN,2017/03,"OUTGOING~0~NUSE~127625172~02032017~000138~17138579~ORIGIN~ALTA PRIORIDAD~INF DE UNA RI?A SJ 926 AGREDIENDO A LA DUE?A DE LA CASA QUE ES UNA MUJER DE LA TERCERA EDAD ~BAJA~,BOGOTA D.C.,ANTONIO NARI?O,LA FRAGUA,,KR 28 15A SUR,25,,,~-74.1064~4.59266~934, 611M~DIRECTION~MESSAGEID~ORIGORGANISATION~ORIGINCIDENTURN~ORIGINCIDENTDATE~ORIGINCIDENTTIME~CALLERNUMBER~CALLORIGIN~PRIORITY~DESCRIPTION~INCSECURITY~LOCATION~EASTING~NORTHING~TYPE~~"
1,NU_4_477409685,14-MAY-18,1731,2018,5,10,ENGATIVA,UPZ30,BOYACA REAL,005603,BOYACA,2824,S.C. BOYACA,934,934 - RIÑA,4.688754,-74.097774,KR 74A 71A 78,NaN,2018/05,"OUTGOING~150~NUSE~477409685~14052018~172849~3204656138~ORIGIN~ALTA PRIORIDAD~RI?A // TIA Y MENOR D E EDAD //NO ARMAS NO HERIDOS //ESTA DROGADO LLAMO HACE 10 MINUTOS NO LLEGA LA AYUDA //~ENTRADA DE LA ALCALDIA DE BOYACA REAL ~BAJA~,,ENGATIVA,BOYACA,,KR 74A 71A,78,AP 2,,~0~0~934, 926M, 611M~DIRECTION~MESSAGEID~ORIGORGANISATION~ORIGINCIDENTURN~ORIGINCIDENTDATE~ORIGINCIDENTTIME~CALLERNUMBER~CALLORIGIN~PRIORITY~DESCRIPTION~INCADDNLINFO~INCSECURITY~LOCATION~EASTING~NORTHING~TYPE~~"
68,NU_5_78285171,01-FEB-17,1545,2017,2,8,KENNEDY,UPZ44,AMERICAS,004501,HIPOTECHO,1265,LAS AMERICAS_,934,934 - RIÑA,4.627819,-74.137076,CL 5 76,CREADO,2017/02,OUTGOING~CURRENT DATA: PRIORITY[ALTA PRIORIDAD]~DIRECTION~UPDATE TEXT~~
69,NU_5_78285171,01-FEB-17,1545,2017,2,8,KENNEDY,UPZ44,AMERICAS,004501,HIPOTECHO,1265,LAS AMERICAS_,934,934 - RIÑA,4.627819,-74.137076,CL 5 76,CREADO,2017/02,OUTGOING~CURRENT DATA: CALLERNUMBER[3185323657]~DIRECTION~UPDATE TEXT~~
70,NU_9_181526173,07-FEB-17,820,2017,2,10,ENGATIVA,UPZ26,LAS FERIAS,005405,LAS FERIAS OCCIDENTAL,2332,LAS FERIAS OCCIDENTAL,934,934 - RIÑA,4.685589,-74.086658,CL 75 69H,CREADO,2017/02,"OUTGOING~55~NUSE~181526173~07022017~082003~3114786341~ORIGIN~ALTA PRIORIDAD~VARIOS VHS DE CLARO MAL ESTACIONADOS DE PLACAS TAM 904 //ZZR 490 //TEO 587 Y ESTAN DISCUTIENDO NO HAY HERIDOS NI ARMAS ~COLEGIO LA PRESENTACION LA FERIAS //SAN JOAQUIN ~BAJA~,BOGOTA D.C.,ENGATIVA,LAS FERIAS,,CL 75 69H,,,,~-74.0864~4.6852~934, 914~DIRECTION~MESSAGEID~ORIGORGANISATION~ORIGINCIDENTURN~ORIGINCIDENTDATE~ORIGINCIDENTTIME~CALLERNUMBER~CALLORIGIN~PRIORITY~DESCRIPTION~INCADDNLINFO~INCSECURITY~LOCATION~EASTING~NORTHING~TYPE~~"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49743,NUSE_289195,15-JAN-19,656,2019,1,16,PUENTE ARANDA,UPZ108,ZONA INDUSTRIAL,006204,ESTACION CENTRAL,3193,ESTACION CENTRAL,934,934 - RIÑA,4.618312,-74.088589,KR 30 19,CERRADO,2019/01,"OUTGOING~CURRENT DATA: LOCATION[,,PUENTE ARANDA,,,KR 30 19,,,,]~DIRECTION~UPDATE TEXT~~"
49744,NUSE_289195,15-JAN-19,656,2019,1,16,PUENTE ARANDA,UPZ108,ZONA INDUSTRIAL,006204,ESTACION CENTRAL,3193,ESTACION CENTRAL,934,934 - RIÑA,4.618312,-74.088589,KR 30 19,CERRADO,2019/01,"OUTGOING~CURRENT DATA: DESCRITPION[SENTIDO NORTE- SUR, FRENTE CENTRO COMERCIAL CALIMA ]~DIRECTION~UPDATE TEXT~~"
49745,NUSE_289195,15-JAN-19,656,2019,1,16,PUENTE ARANDA,UPZ108,ZONA INDUSTRIAL,006204,ESTACION CENTRAL,3193,ESTACION CENTRAL,934,934 - RIÑA,4.618312,-74.088589,KR 30 19,CERRADO,2019/01,"OUTGOING~87~NUSE~289195~15012019~065601~34-9~ORIGIN~ALTA PRIORIDAD~CHOQUE MULTIPLE ENTRE VEHICULOS, ESTAN FORMENTANDO RI?A, SE SOLICITA APOYO DE LA PONAL Y UNIDAD DE TRANSITO DE CARACTER URGENTEEE~SENTIDO NORTE- SUR, FRENTE CENTRO COMERCIAL CALIMA ~BAJA~,,PUENTE ARANDA,,,KR 30 19,,,,~0~0~942, 934~DIRECTION~MESSAGEID~ORIGORGANISATION~ORIGINCIDENTURN~ORIGINCIDENTDATE~ORIGINCIDENTTIME~CALLERFORENAME~CALLORIGIN~PRIORITY~DESCRIPTION~INCADDNLINFO~INCSECURITY~LOCATION~EASTING~NORTHING~TYPE~~"
49746,NUSE_122658195,27-JAN-19,74

In [113]:
merged_nuse.to_csv(r'/Users/anamaria/Desktop/dev/security_project/datasets/merged_nuse.csv',index=None)

# 2. Rebuild missing data

In [75]:
def find_char(df, column_name, char=''):
    array_char = np.where(df[column_name] == char)
    if array_char[0].size > 0:
        return array_char[0]
    else:
        return 0

In [114]:
print("Tamaño:",merged_nuse.shape)
print("size: ",merged_nuse.size)
print("Porcentaje Celdas Vacias:",merged_nuse.isna().mean().mean())
print("Porcentaje Celdas con valor '-':",(merged_nuse == "-").mean().mean())
print("Filas duplicadas",merged_nuse.duplicated().sum())

Tamaño: (1108835, 21)
size:  23285535
Porcentaje Celdas Vacias: 1.7178046370847824e-07
Porcentaje Celdas con valor '-': 0.002191188649949421
Filas duplicadas 0


In [115]:
pd.DataFrame({"Tipo de dato":merged_nuse.dtypes.values,
              "Celdas con valor '-'":(merged_nuse == '-').sum().values,
              "Celdas con valor ''":(merged_nuse == '').sum().values,
              "Celdas con valor ' '":(merged_nuse == ' ').sum().values,
              "Celdas vacías": merged_nuse.isna().sum().values},
             index=merged_nuse.columns)

,Tipo de dato,Celdas con valor '-',Celdas con valor '',Celdas con valor ' ',Celdas vacías
STR_NUMERO_INTERNO,object,0,0,0,0
FECHA,object,0,0,0,0
HORA,int64,0,0,0,0
ANIO,int64,0,0,0,0
MES,int64,0,0,0,0
COD_LOCALIDAD,int64,0,0,0,0
LOCALIDAD,object,0,0,0,0
COD_UPZ,object,2453,0,0,0
UPZ,object,2453,0,0,0
COD_SEC_CATAST,object,2453,0,0,0


In [121]:
df1 = merged_nuse.loc[merged_nuse['COD_UPZ'] == '-']
df2 = merged_nuse.loc[merged_nuse['UPZ'] == '-']
df3 = merged_nuse.loc[merged_nuse['COD_SEC_CATAST'] == '-']
df4 = merged_nuse.loc[merged_nuse['SEC_CATASTRAL'] == '-']
df5 = merged_nuse.loc[merged_nuse['COD_BARRIO'] == '-']
df6 = merged_nuse.loc[merged_nuse['BARRIO'] == '-']

In [122]:
df1.equals(df2) and df1.equals(df3) and df1.equals(df4) and df1.equals(df5) and df1.equals(df6)

True

In [123]:
df_direccion=df1.loc[df1['STR_DIRECCION_INCIDENTE'] != '-']
df_direccion.head()

,STR_NUMERO_INTERNO,FECHA,HORA,ANIO,MES,COD_LOCALIDAD,LOCALIDAD,COD_UPZ,UPZ,COD_SEC_CATAST,SEC_CATASTRAL,COD_BARRIO,BARRIO,TIPO_UNICO,TIPO_DETALLE,LATITUD,LONGITUD,STR_DIRECCION_INCIDENTE,ESTADO_INCIDENTE,PERIODO_TS,LOG_TEXT
43743,NU_8_19979173,02-JAN-17,56,2017,1,99,SIN LOCALIZACION,-,-,-,-,-,-,934,934 - RIÑA,-1.0,-1.0,CL 37A 44 ESTE 67,CERRADO,2017/01,"OUTGOING~285~NUSE~19979173~02012017~005616~3209197337~DIEGO MU?OS ~ORIGIN~ALTA PRIORIDAD~RED DE APOYO // RI?A DE PAREJA DENTRO DE LA VIVIENDA // SUJETO AGRESIENDO A LA MUJER // NO SABE DE ARMAS O HERIDOS // LA MUJER ESTA ENCERRADA EN EL BA?O ~SECTOR BUENOS ~BAJA~,,,CIUDADELA SUCRE,,CL 37A 44 ESTE,67,,,~0~0~934, 611M~DIRECTION~MESSAGEID~ORIGORGANISATION~ORIGINCIDENTURN~ORIGINCIDENTDATE~ORIGINCIDENTTIME~CALLERNUMBER~CALLERFORENAME~CALLORIGIN~PRIORITY~DESCRIPTION~INCADDNLINFO~INCSECURITY~LOCATION~EASTING~NORTHING~TYPE~~"
44837,NU_8_36613173,11-JAN-17,34,2017,1,99,SIN LOCALIZACION,-,-,-,-,-,-,934,934 - RIÑA,-1.0,-1.0,CL 14,CERRADO,2017/01,"OUTGOING~109~NUSE~36613173~11012017~003412~3132664796~ORIGIN~ALTA PRIORIDAD~VARIOS JOVENES SE ESTAN AGREDIENDO Y HACEN MUCHO RUIDO NO SABE DE ARMAS O HERIDOS ~RED DE APOYO POR LA FABRICA DE AGUILA ROJA AL PIE DE LA LECHONERIA EN LA CASA DE PORTONES CAFE EN LOS LOTES DESOCUPADOS CERCA AL BARRIO SAN DIEGO ~BAJA~,,MADRID CUNDINAMARCA,BOSQUES DE MADRID,,CL 14,,,,~0~0~934~DIRECTION~MESSAGEID~ORIGORGANISATION~ORIGINCIDENTURN~ORIGINCIDENTDATE~ORIGINCIDENTTIME~CALLERNUMBER~CALLORIGIN~PRIORITY~DESCRIPTION~INCADDNLINFO~INCSECURITY~LOCATION~EASTING~NORTHING~TYPE~~"
49576,NU_4_192808173,10-FEB-17,739,2017,2,99,SIN LOCALIZACION,-,-,-,-,-,-,934,934 - RIÑA,-1.0,-1.0,VIA BOGOTA- GIRARDOT,CERRADO,2017/02,"OUTGOING~139~NUSE~192808173~10022017~073935~3193390792~ORIGIN~ALTA PRIORIDAD~CONDUCTOR DE BUS INTERMUNICIPAL DE LA EMPRESA TRANSPORTES RAPIDO GIRARDOT PLACAS SXU-601 #INTERNO 232 AGRESIVO CON LIS USUARIOS// EL BUS MECANICAMENTE ESTA EN MALAS CONDICIONES POR POCO SE ESTRELLA ~RED DE APOYO~BAJA~,,ALTO DE LA ROSA-CUNDINAMA,,,VIA BOGOTA- GIRARDOT,,,,~0~0~934~DIRECTION~MESSAGEID~ORIGORGANISATION~ORIGINCIDENTURN~ORIGINCIDENTDATE~ORIGINCIDENTTIME~CALLERNUMBER~CALLORIGIN~PRIORITY~DESCRIPTION~INCADDNLINFO~INCSECURITY~LOCATION~EASTING~NORTHING~TYPE~~"
51353,NU10_242419171,19-FEB-17,1145,2017,2,99,SIN LOCALIZACION,-,-,-,-,-,-,934,934 - RIÑA,-1.0,-1.0,CL 43F 27 ESTE 43,CERRADO,2017/02,"OUTGOING~326~NUSE~242419171~19022017~114544~3144895444~AULINO ALVAREZ ~ORIGIN~ALTA PRIORIDAD~RED DE APOYO // INF DE DOS SUJ EN RI?A CON BOTELLAS HAY UN HERIDO~BAJA~,,SOACHA /CUNDINAMARCA,LUIS CARLOS GALAN II SECTOR,,CL 43F 27 ESTE,43,,,~0~0~910, 934~DIRECTION~MESSAGEID~ORIGORGANISATION~ORIGINCIDENTURN~ORIGINCIDENTDATE~ORIGINCIDENTTIME~CALLERNUMBER~CALLERFORENAME~CALLORIGIN~PRIORITY~DESCRIPTION~INCSECURITY~LOCATION~EASTING~NORTHING~TYPE~~"
54232,NU_6_264756171,09-MAR-17,2042,2017,3,99,SIN LOCALIZACION,-,-,-,-,-,-,934,934 - RIÑA,-1.0,-1.0,DG 49D SUR 13H,CERRADO,2017/03,"OUTGOING~295~NUSE~264756171~09032017~204232~3182990520~ORIGIN~ALTA PRIORIDAD~INFO DE UN GRUPO DE CINCUENTA HINCHAS DE MILLONARIOS GENERANDO RI?A Y CONSUMIENDO 922 // NO SABE DE ARMAS ~BAJA~,BOGOTA,RAFAEL URIBE,MARCO FIDEL SUAREZ I,,DG 49D SUR 13H,,,,~-74.12129~4.567053~968, 934~DIRECTION~MESSAGEID~ORIGORGANISATION~ORIGINCIDENTURN~ORIGINCIDENTDATE~ORIGINCIDENTTIME~CALLERNUMBER~CALLORIGIN~PRIORITY~DESCRIPTION~INCSECURITY~LOCATION~EASTING~NORTHING~TYPE~~"


In [124]:
df_direccion.to_csv(r'/Users/anamaria/Desktop/dev/security_project/datasets/df_direccion.csv',index=None)

In [99]:
log_field=pd.read_csv('/Users/anamaria/Desktop/log_text.csv',delimiter="~")

In [100]:
log_field.head()

,OUTGOING,238,NUSE,273313172,05042017,204013,16958322,ORIGIN,ALTA PRIORIDAD,REPORTA QUE LA HERMANA MALTRADA A DOS HIJAS QUE TIENE UNA DE 3 A?OS Y UNA DE A?O Y MEDIO INF QUE LAS ESTA GOLPEANDO EN ESTE MOMENTO,BARRIO LA GAITANA,BAJA,",BOGOTA D.C.,SUBA,PUERTA DEL SOL,,KR 112B 139,19,,,",-74.103963,4.743626,611,DIRECTION,MESSAGEID,ORIGORGANISATION,ORIGINCIDENTURN,ORIGINCIDENTDATE,ORIGINCIDENTTIME,CALLERNUMBER,CALLORIGIN,PRIORITY,DESCRIPTION,INCADDNLINFO,INCSECURITY,LOCATION,EASTING,NORTHING,TYPE,Unnamed: 32,Unnamed: 33
0,OUTGOING,1,NUSE,274722172,5042017.0,230149.0,3.224844e+09,ORIGIN,ALTA PRIORIDAD,INF DE UNA NI?A ALPARECER 7 A?OS DENTRO DE VIVIENDA GRITANDO/CREEN QUE STA SOLA,CASA DE COLOR FACHADA LADRILLO /DONDE ESTA LA CERCA ELECTRICA PUERTA COLOR VERDE /DONDE ESTA EL GRAFITO LA MADRE PARA QUIEN,BAJA,",,BOSA,POTRERITOS,,CL 81A SUR 91,,,,",0.0,0,611,DIRECTION,MESSAGEID,ORIGORGANISATION,ORIGINCIDENTURN,ORIGINCIDENTDATE,ORIGINCIDENTTIME,CALLERNUMBER,CALLORIGIN,PRIORITY,DESCRIPTION,INCADDNLINFO,INCSECURITY,LOCATION,EASTING,NORTHING,TYPE,NaN,NaN
1,OUTGOING,104,NUSE,89445194,11012019.0,122154.0,3.134160e+09,ORIGIN,ALTA PRIORIDAD,SJ MALTRATANDO A UNA MUJER DISCAPACITADA / NO HERIDOS NO ARMAS,BAJA,",,SOACHA CUNDINAMARCA,BELLA VISTA,,CL 39I 42 ESTE,52,,,",0,0.0,"934, 611M",DIRECTION,MESSAGEID,ORIGORGANISATION,ORIGINCIDENTURN,ORIGINCIDENTDATE,ORIGINCIDENTTIME,CALLERNUMBER,CALLORIGIN,PRIORITY,DESCRIPTION,INCSECURITY,LOCATION,EASTING,NORTHING,TYPE,NaN,NaN,NaN,NaN
2,OUTGOING,240,NUSE,1152151173,9092017.0,124021.0,3.143945e+09,ORIGIN,ALTA PRIORIDAD,INFORMA QUE HAY UNA SE?ORA QUE LE ESTA TIRANDO PIEDRAS A LOS HIJOS ESTA AGRESIVA,RED DE APOYO,BAJA,",,SOACHA / CUNDINAMARCA,MINUTO,,CL 58 23 ESTE,81,,,",0.0,0,611,DIRECTION,MESSAGEID,ORIGORGANISATION,ORIGINCIDENTURN,ORIGINCIDENTDATE,ORIGINCIDENTTIME,CALLERNUMBER,CALLORIGIN,PRIORITY,DESCRIPTION,INCADDNLINFO,INCSECURITY,LOCATION,EASTING,NORTHING,TYPE,NaN,NaN
3,OUTGOING,129,NUSE,1080892171,12102017.0,102547.0,3.125665e+09,ORIGIN,ALTA PRIORIDAD,"INF. BEBE DESDE HACE TRES DIAS LLORANDO, PADRES CONSUMIDORES",BAJA,",,SOACHA,LOS ROBLES,,CL 46 ESTE 40,30,,,",0,0.0,611,DIRECTION,MESSAGEID,ORIGORGANISATION,ORIGINCIDENTURN,ORIGINCIDENTDATE,ORIGINCIDENTTIME,CALLERNUMBER,CALLORIGIN,PRIORITY,DESCRIPTION,INCSECURITY,LOCATION,EASTING,NORTHING,TYPE,NaN,NaN,NaN,NaN
4,OUTGOING,CURRENT DATA: CALLERNUMBER[3125664621],DIRECTION,UPDATE TEXT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [101]:
data.LOCALIDAD.unique()

array(['ANTONIO NARIÑO', 'ENGATIVA', 'KENNEDY', 'SUBA',
       'RAFAEL URIBE URIBE', 'USAQUEN', 'TUNJUELITO', 'CHAPINERO', 'BOSA',
       'USME', 'BARRIOS UNIDOS', 'SAN CRISTOBAL', 'CIUDAD BOLIVAR',
       'FONTIBON', 'CANDELARIA', 'SIN LOCALIZACION', 'TEUSAQUILLO',
       'PUENTE ARANDA', 'LOS MARTIRES', 'SANTA FE'], dtype=object)

In [107]:
data.loc[data['BARRIO'] == 'POTRERITOS']

,STR_NUMERO_INTERNO,FECHA,HORA,ANIO,MES,COD_LOCALIDAD,LOCALIDAD,COD_UPZ,UPZ,COD_SEC_CATAST,SEC_CATASTRAL,COD_BARRIO,BARRIO,TIPO_UNICO,TIPO_DETALLE,LATITUD,LONGITUD,STR_DIRECCION_INCIDENTE,ESTADO_INCIDENTE,PERIODO_TS,LOG_TEXT
6932,NU_6_98586173,16-JAN-17,1822,2017,1,19,CIUDAD BOLIVAR,UPZ68,EL TESORO,002533,SOTAVENTO,1502,POTRERITOS,611M,611M - MALTRATO A MUJER,4.543807,-74.138477,CL 73B SUR 14 11,CERRADO,2017/01,"OUTGOING~125~NUSE~98586173~16012017~182217~3112243594~ELIANA SU?IGA ~ORIGIN~ALTA PRIORIDAD~INFO DE CU?ADO AL AGREDIO / EL SJT ROMPIO LA PUERTA DEL SITIO / NO ARMA SNO HERIDOS ESTAN EN PRESENCIA DE MENORES DE EDAD / ~DIRECCION NUEVA / AL FRENTE DE LA ESTACION DE ESSO MOBIL / EN EL SEGUNDO PISO DEL NEGOCIO EL TORNILLERO DEL SUR ~BAJA~,,USME,SANTA LIBRADA,,CL 73B SUR 14,11,,,~0~0~934, 912, 611M~DIRECTION~MESSAGEID~ORIGORGANISATION~ORIGINCIDENTURN~ORIGINCIDENTDATE~ORIGINCIDENTTIME~CALLERNUMBER~CALLERFORENAME~CALLORIGIN~PRIORITY~DESCRIPTION~INCADDNLINFO~INCSECURITY~LOCATION~EASTING~NORTHING~TYPE~~"
21258,NU_6_1259196172,30-OCT-17,2226,2017,10,19,CIUDAD BOLIVAR,UPZ68,EL TESORO,002533,SOTAVENTO,1502,POTRERITOS,611M,611M - MALTRATO A MUJER,4.543807,-74.138477,CL 73C SUR 14R 51,CERRADO,2017/10,"OUTGOING~140~NUSE~1259196172~30102017~222646~3203902664~JULIAN RODRIGUEZ ~ORIGIN~ALTA PRIORIDAD~RI?A INTRAFAMILIAR EN LA VIVIENDA // LOS HIJOS LE PEGAN A LA HERMANA Y AGREDEN A LA MAMA // NO ARMAS NI HERIDOS~BAJA~,,USME,SAN LUIS,,CL 73C SUR 14R,51,,,~0~0~934, 611M~DIRECTION~MESSAGEID~ORIGORGANISATION~ORIGINCIDENTURN~ORIGINCIDENTDATE~ORIGINCIDENTTIME~CALLERNUMBER~CALLERFORENAME~CALLORIGIN~PRIORITY~DESCRIPTION~INCSECURITY~LOCATION~EASTING~NORTHING~TYPE~~"
22329,NU11_1459678173,21-NOV-17,1059,2017,11,19,CIUDAD BOLIVAR,UPZ68,EL TESORO,002533,SOTAVENTO,1502,POTRERITOS,611M,611M - MALTRATO A MUJER,4.543807,-74.138477,CL 73B SUR 14P 66,CERRADO,2017/11,"OUTGOING~318~NUSE~1459678173~21112017~105923~3132323283~ANGELICA ~ORIGIN~ALTA PRIORIDAD~INFO DE HOMBRE QUE GOLPEA A MUJER LLAMANTE DENTRO DE VIVIENDA / PERSONA CON ARMA BLANCA // NO HERIDOS ~BAJA~,BOGOTA,USME,EL CORTIJO,NE,CL 73B SUR 14P,66,,,~-74.13627~4.543692~934, 611M~DIRECTION~MESSAGEID~ORIGORGANISATION~ORIGINCIDENTURN~ORIGINCIDENTDATE~ORIGINCIDENTTIME~CALLERNUMBER~CALLERFORENAME~CALLORIGIN~PRIORITY~DESCRIPTION~INCSECURITY~LOCATION~EASTING~NORTHING~TYPE~~"
24092,NU_5_1533948172,23-DEC-17,1805,2017,12,19,CIUDAD BOLIVAR,UPZ68,EL TESORO,002533,SOTAVENTO,1502,POTRERITOS,611M,611M - MALTRATO A MUJER,4.543807,-74.138477,CL 73C SUR 14R 44,CERRADO,2017/12,"OUTGOING~28~NUSE~1533948172~23122017~180524~3124146679~VIVIANA ~ORIGIN~MEDIA PRIORIDAD~SJ EN 926* FORMANDO RI?A INTRAFAMILIAR //ESTA MUY AGRESIVO PEGANDOLE A LA MUJER Y CU?ADA AMBARAZADA / HAY MENORES PRESENTES **~DIRECCION NUEVA **NO HERIDOS POR EL MOMENTO **~BAJA~,,USME,LA REGADERA,,CL 73C SUR 14R,44,,,~0~0~934, 611M~DIRECTION~MESSAGEID~ORIGORGANISATION~ORIGINCIDENTURN~ORIGINCIDENTDATE~ORIGINCIDENTTIME~CALLERNUMBER~CALLERFORENAME~CALLORIGIN~PRIORITY~DESCRIPTION~INCADDNLINFO~INCSECURITY~LOCATION~EASTING~NORTHING~TYPE~~"
24189,NU_1_1554934172,24-DEC-17,2211,2017,12,19,CIUDAD BOLIVAR,UPZ68,EL TESORO,002533,SOTAVENTO,1502,POTRERITOS,611M,611M - MALTRATO A MUJER,4.543348,-74.138589,CL 74 SUR 17A 56,CERRADO,2017/12,"OUTGOING~113~NUSE~1554934172~24122017~221157~3163814275~ORIGIN~ALTA PRIORIDAD~SUJETO LE AGREDE LA QUIERE GOLPIAR ELLA TIENE MEDIDA DE PROTECCION ///NO ARMAS NO HERIDOS //~POR LAS CANCHAS DE POTRERITOS ~BAJA~,BOGOTA D.C.,CIUDAD BOLIVAR,POTRERITOS,,CL 74 SUR 17A,56,,,~-74.13866~4.54335~934, 611M~DIRECTION~MESSAGEID~ORIGORGANISATION~ORIGINCIDENTURN~ORIGINCIDENTDATE~ORIGINCIDENTTIME~CALLERNUMBER~CALLORIGIN~PRIORITY~DESCRIPTION~INCADDNLINFO~INCSECURITY~LOCATION~EASTING~NORTHING~TYPE~~"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22770,NU_6_66014194,08-JAN-19,1440,2019,1,19,CIUDAD BOLIVAR,UPZ68,EL TESORO,002533,SOTAVENTO,1502,POTRERITOS,934,934 - RIÑA,4.543807,-74.138477,CL 73B BIS SUR 14 41,CERRADO,20

In [102]:
sorted(data.LOCALIDAD.unique())

['ANTONIO NARIÑO',
 'BARRIOS UNIDOS',
 'BOSA',
 'CANDELARIA',
 'CHAPINERO',
 'CIUDAD BOLIVAR',
 'ENGATIVA',
 'FONTIBON',
 'KENNEDY',
 'LOS MARTIRES',
 'PUENTE ARANDA',
 'RAFAEL URIBE URIBE',
 'SAN CRISTOBAL',
 'SANTA FE',
 'SIN LOCALIZACION',
 'SUBA',
 'TEUSAQUILLO',
 'TUNJUELITO',
 'USAQUEN',
 'USME']